In [16]:
from pathlib import Path

import pandas as pd
import numpy as np

from src import constants, utils

In [17]:
def get_angle(x1, y1, x2, y2):
    dy = y2 - y1
    dx = x2 - x1
    angle = np.degrees(np.arctan2(dy, dx))
    return angle

In [18]:
def load_df(path):
    df = pd.read_csv(path, index_col=0)
    df = df[df.pop("condition").str.contains("Spinal")]
    df["instance_number"] = df["instance_number"].replace(-1, pd.NA).astype("Int64").bfill()
    df["side"] = df["side"].map({"L": "left", "R": "right"})
    # df = df.pivot(index=constants.BASIC_COLS + ["level"], columns="side", values=["relative_x", "relative_y"])
    return df


df = load_df("data/coords_rsna_improved.csv")
df

,series_id,relative_x,relative_y,side,instance_number,study_id,level
20,370109,0.390625,0.261719,left,9,1217477368,L1/L2
21,370109,0.564182,0.282331,right,9,1217477368,L1/L2
22,370109,0.367188,0.398438,left,9,1217477368,L2/L3
23,370109,0.546010,0.413573,right,9,1217477368,L2/L3
24,370109,0.339844,0.531250,left,9,1217477368,L3/L4
...,...,...,...,...,...,...,...
58730,4294540297,0.511282,0.571751,right,10,916362094,L3/L4
58731,4294540297,0.308594,0.667969,left,9,916362094,L4/L5
58732,4294540297,0.516697,0.671029,right,9,916362094,L4/L5
58733,4294540297,0.328125,0.796875,left,10,916362094,L5/S1


In [19]:
meta = utils.load_meta(with_center=False, with_normal=False, with_relative_position=False)[constants.BASIC_COLS + ["Rows", "Columns"]]

In [20]:
df = df.merge(meta, how="inner", on=constants.BASIC_COLS)

In [21]:
df["x"] = df.pop("relative_x") * df.pop("Columns")
df["y"] = df.pop("relative_y") * df.pop("Rows")

df

,series_id,side,instance_number,study_id,level,x,y
0,370109,left,9,1217477368,L1/L2,200.000000,134.000000
1,370109,right,9,1217477368,L1/L2,288.861071,144.553322
2,370109,left,9,1217477368,L2/L3,188.000000,204.000000
3,370109,right,9,1217477368,L2/L3,279.557010,211.749318
4,370109,left,9,1217477368,L3/L4,174.000000,272.000000
...,...,...,...,...,...,...,...
19720,4294540297,right,10,916362094,L3/L4,229.054151,256.144404
19721,4294540297,left,9,916362094,L4/L5,138.250000,299.250000
19722,4294540297,right,9,916362094,L4/L5,231.480144,300.620939
19723,4294540297,left,10,916362094,L5/S1,147.000000,357.000000


In [22]:
df.to_csv("data/t2_coordinates.csv", index=False)